In [32]:
import re
import httpx
import os
from dashscope import Generation
from http import HTTPStatus

In [33]:
from qwen import QOWCaller
client = QOWCaller()

In [34]:
class Agent:
    def __init__(self, system="", model_name="qwen-max"):
        self.system = system
        self.model_name = model_name
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        response = Generation.call(
            model=self.model_name,
            messages=self.messages,
            result_format="message",
        )
        if response.status_code != HTTPStatus.OK:
            print(
                "Request id: %s, Status code: %s, error code: %s, error message: %s"
                % (
                    response.request_id,
                    response.status_code,
                    response.code,
                    response.message,
                )
            )

        return response.output.choices[0].message.content

In [35]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [36]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [37]:
abot = Agent(prompt)
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should find out the weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [38]:

result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [39]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [40]:
abot(next_prompt)

"Answer: A Toy Poodle's average weight is 7 lbs."

In [41]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [42]:
import re
action_re = re.compile('^Action: (\w+): (.*)$')

In [43]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [45]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find out the average weight of a Border Collie and a Scottish Terrier, then add those two weights together.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I know the average weight of a Border Collie, I will find out the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have the weights of both breeds, now I just need to add them together to get the combined weight of the two dogs.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of your Border Collie and Scottish Terrier is 57 lbs.


In [46]:
search_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search_website_query
e.g. search_website_query: what is the weather like in Dalian
return the current weather in Dalian

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary



Example session:

Question: What is the weather like in HangZhou?
Thought: I should look the weather in HangZhou using search_website_query
Action: search_website_query: HangZhou
PAUSE
Action: search_website_query: HangZhou 
PAUSE

You will be called again with this:

Observation: the The weather in Hangzhou currently is cloudy with a temperature of 22.5°C (72.5°F). The wind speed is 6.8 kph coming from the east-southeast direction. The humidity level is at 76%, and the visibility is 10.0 km. 

You then output: Hangzhou cloudy with a temperature of 22.5°C (72.5°F). The wind speed is 6.8 kph coming from the east-southeast direction. The humidity level is at 76%, and the visibility is 10.0 km.

Answer: the current weather in Hangzhou is cloudy with a temperature of 22.5°C (72.5°F). The wind speed is 6.8 kph coming from the east-southeast direction. The humidity level is at 76%, and the visibility is 10.0 km.
""".strip()

In [61]:
def calculate(what):
    return eval(what)


def search_website_query(query):
    # print("Searching website")
    # run search
    result = client.search(query, include_answer=True)
    # print the answer
    return result["answer"]


search_known_actions = {"calculate": calculate, "search_website_query": search_website_query}

In [48]:
from tavily import TavilyClient

In [63]:
# client = TavilyClient(api_key=os.environ.get("TRAVILY_API_KEY"))
# result = client.search("What is the weather like in Hangzhou?",
#                        include_answer=True)

# # print the answer
# result["answer"]
search_website_query("what is the weather like in dalian?")

'The current weather in Dalian, Liaoning, China is sunny with a temperature of 32.2°C (90.0°F). The wind speed is 34.9 kph coming from the west, and the humidity is at 43%. The weather feels like 36.1°C (96.9°F).'

In [51]:
search_bot = Agent(search_prompt)
result = search_bot("what is the weather like in Dalian")
print(result)

Thought: I should look up the weather in Dalian using the search_website_query action.
Action: search_website_query: what is the weather like in Dalian
PAUSE


In [55]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(search_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in search_known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = search_known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [65]:
question = """between dalian and hangzhou which weather is better today?"""
query(question)

Thought: I need to check the current weather in both Dalian and Hangzhou to compare them. I will start with Dalian.
Action: search_website_query: current weather in Dalian
PAUSE

You will be called again with this:


 -- running search_website_query current weather in Dalian
Observation: The current weather in Dalian, China is sunny with a temperature of 32.2°C (90.0°F). The wind speed is 34.9 kph coming from the west, with a humidity of 43%.
Thought: Now that I have the weather for Dalian, I should look up the current weather in Hangzhou to make a comparison.
Action: search_website_query: current weather in Hangzhou
PAUSE

You will be called again with this:


 -- running search_website_query current weather in Hangzhou
Observation: The current weather in Hangzhou, China is sunny with a temperature of 25°C (77°F). The wind is coming from the southeast at 11.2 km/h (6.9 mph) with a humidity level of 69%. The visibility is 10.0 km (6.0 miles) and the UV index is 6.0.
Thought: Having obs

In [66]:
question = """which mountain is the highest mountain in the world?"""
query(question)

Thought: I should search the web to find out which mountain is the highest in the world.
Action: search_website_query: highest mountain in the world
PAUSE
 -- running search_website_query highest mountain in the world
Observation: Mount Everest is the highest mountain in the world, standing at 8,848.86 meters (29,031.7 feet) above sea level.
Answer: The highest mountain in the world is Mount Everest, with an elevation of 8,848.86 meters (29,031.7 feet) above sea level.
